In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load dataset
data = load_dataset("Kobi-01/tamil_agriculture_QA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/532 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.58M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/574k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/574k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15830 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3392 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3393 [00:00<?, ? examples/s]

In [ ]:
# Load teacher model and tokenizer
teacher_model_path = "abhinand/tamil-llama-7b-instruct-v0.1"
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_path)
teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_path).to(device)

tokenizer_config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

...

pytorch_model-00005-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
# Use the same tokenizer for the student model
student_tokenizer = teacher_tokenizer
student_model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m").to(device)

# Resize the student model's token embeddings to match the teacher tokenizer's vocabulary size
student_model.resize_token_embeddings(len(teacher_tokenizer))

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Embedding(47957, 768)

In [ ]:
def preprocess_function(examples):
    inputs = [f"Q: {q} A: {a}" for q, a in zip(examples["question"], examples["answer"])]
    tokenized_inputs = student_tokenizer(inputs, padding='max_length', truncation=True, max_length=512)
    return tokenized_inputs

# Tokenize the dataset
tokenized_data = data.map(preprocess_function, batched=True)
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask'])

Map:   0%|          | 0/15830 [00:00<?, ? examples/s]

Map:   0%|          | 0/3392 [00:00<?, ? examples/s]

Map:   0%|          | 0/3393 [00:00<?, ? examples/s]

In [ ]:
# Function to evaluate model performance
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Forward pass through the model
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Get predictions from logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            all_preds.extend(preds)

            # Collect the true labels
            labels = batch['labels'].cpu().numpy()
            all_labels.extend(labels)

    return all_preds, all_labels


In [ ]:
# Distillation loss function
def distillation_loss(student_logits, teacher_logits, temperature, alpha):
    soft_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    student_soft = nn.functional.log_softmax(student_logits / temperature, dim=-1)
    distill_loss = nn.functional.kl_div(student_soft, soft_targets, reduction='batchmean') * (temperature ** 2)
    return distill_loss


In [ ]:
# Hyperparameters
batch_size = 6
lr = 2e-5
num_epochs = 3
temperature = 2.0
alpha = 0.7

# Optimizer
optimizer = optim.Adam(student_model.parameters(), lr=lr)

# Create dataloaders
train_dataloader = DataLoader(tokenized_data['train'], batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(tokenized_data['test'], batch_size=batch_size)


In [ ]:
# Training loop
student_model.train()
for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        try:
            # Move inputs to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Debugging: Assert input shapes are correct
            assert input_ids.shape[1] <= 512, "Input sequence length exceeds max_length"
            assert attention_mask.shape == input_ids.shape, "Mismatch between input_ids and attention_mask shapes"

            # Forward pass through teacher model (without gradients)
            with torch.no_grad():
                teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)
                teacher_logits = teacher_outputs.logits

            # Forward pass through student model
            student_outputs = student_model(input_ids, attention_mask=attention_mask)
            student_logits = student_outputs.logits

            # Compute distillation loss
            loss = distillation_loss(student_logits, teacher_logits, temperature, alpha)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging
            if step % 10 == 0:
                print(f"Epoch {epoch + 1}, Step {step}, Loss: {loss.item()}")

        except RuntimeError as e:
            print(f"RuntimeError at Step {step}, skipping batch: {e}")
            continue

    print(f"Epoch {epoch + 1} completed with final loss: {loss.item()}")


Epoch 1, Step 0, Loss: 6251.3310546875
RuntimeError at Step 1, skipping batch: CUDA out of memory. Tried to allocate 562.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 138.81 MiB is free. Process 21497 has 39.42 GiB memory in use. Of the allocated memory 37.49 GiB is allocated by PyTorch, and 1.43 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
RuntimeError at Step 2, skipping batch: CUDA out of memory. Tried to allocate 130.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 6.81 MiB is free. Process 21497 has 39.55 GiB memory in use. Of the allocated memory 38.34 GiB is allocated by PyTorch, and 729.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable

KeyboardInterrupt: 

In [ ]:
# # Evaluate on test data
# student_model.eval()
# for batch in test_dataloader:
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)

#     with torch.no_grad():
#         student_outputs = student_model(input_ids, attention_mask=attention_mask)

#     print(f"Predictions: {student_outputs.logits.argmax(dim=-1)}")